In [ ]:
model_path = 'saved_models/best_ae_64_tanh_lr1e4_50ep_AdamW.keras'  
model = load_model(
    model_path,
    custom_objects={'r2': r2},
    compile=False
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(),
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError(), r2]
)

In [ ]:
#Evaluation

all_losses, threshold = evaluate_and_detect(model, test_ds)

In [ ]:
#Latent characteristics 
latents = extract_and_save_latents(model, test_ds, output_path="latent_features_test.npy")

In [ ]:
#Reconstruct and evaluate error 
# ————————————————————————————————————
# 1) Configuración inicial
# ————————————————————————————————————
# Ruta al modelo autoencoder entrenado
MODEL_PATH = "saved_models/best_ae_64_tanh_lr1e4_50ep_AdamW.keras"

# ————————————————————————————————————
# 2) Carga del modelo y definición del encoder
# ————————————————————————————————————
# Cargamos el AE completo (sin compilar, para acelerar)
ae = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={'r2': r2},
    compile=False
)

for i, layer in enumerate(ae.layers):
    name = layer.name
    cls  = layer.__class__.__name__

    try:
        shape = layer.output.shape
    except AttributeError:
        # Algunas capas (InputLayer) no tienen .output, pero sí .batch_input_shape
        shape = getattr(layer, 'batch_input_shape', 'unknown')

    print(f"{i:02d}: {name:20s} ({cls:15s}) → {shape}")

In [ ]:
# Construimos el encoder a partir de la capa bottleneck
encoder = tf.keras.Model(
    inputs=ae.input,
    outputs=ae.get_layer("lstm").output
)


In [ ]:
# 3. Codifica tu dataset completo de ciclos de test/train/val
latent_train = encoder.predict(train_ds, verbose=1)
latent_val   = encoder.predict(val_ds,   verbose=1)
latent_test  = encoder.predict(test_ds,  verbose=1)

print("Shapes latentes:", latent_train.shape, latent_val.shape, latent_test.shape)

In [ ]:
# Índices de atributos que queremos reconstruir y medir
ATTR_INDICES = [311, 313, 306]

# Tamaño de lote para evaluación
BATCH_SIZE = 256

test_ds_batched = (
    test_ds
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

# 4) Iteramos para reconstruir y acumular
orig_attrs_batches = []
recon_attrs_batches = []

for batch_data, _ in test_ds_batched:
    # 4.1 Reconstrucción completa (devuelve np.ndarray si ae.predict)
    recon_full = ae.predict(batch_data, verbose=0)  
    
    # 4.2 Convertimos tensores a NumPy
    orig_np  = batch_data.numpy()
    recon_np = recon_full if isinstance(recon_full, np.ndarray) else recon_full.numpy()
    
    # 4.3 Extraemos sólo las columnas de interés
    #     Forma: (batch_size, timesteps, n_attrs)
    orig_sub  = orig_np[..., ATTR_INDICES]
    recon_sub = recon_np[..., ATTR_INDICES]
    
    orig_attrs_batches.append(orig_sub)
    recon_attrs_batches.append(recon_sub)

# 5) Concatenamos todos los batches
orig_attrs  = np.concatenate(orig_attrs_batches, axis=0)   # (N_total, timesteps, n_attrs)
recon_attrs = np.concatenate(recon_attrs_batches, axis=0)  # (N_total, timesteps, n_attrs)

# 6) Calculamos métricas por atributo
#    Promedio sobre muestras y tiempo
mse  = np.mean((recon_attrs - orig_attrs)**2,   axis=(0, 1))
rmse = np.sqrt(mse)
mae  = np.mean(np.abs(recon_attrs - orig_attrs), axis=(0, 1))

# 7) Guardamos y mostramos resultados
np.save("reconstructed_attrs.npy", recon_attrs)
print("Saved reconstructed attributes to reconstructed_attrs.npy\n")

print("Error por atributo:")
for i, attr_idx in enumerate(ATTR_INDICES):
    print(f"  Atributo {attr_idx}:  MSE={mse[i]:.6f},  RMSE={rmse[i]:.6f},  MAE={mae[i]:.6f}")

In [ ]:
#Clustering 

latents = extract_and_save_latents(model, test_ds, output_path="latent_features_test.npy")
labels = KMeans(n_clusters=2).fit_predict(latents)

NameError: name 'extract_and_save_latents' is not defined